In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
import numpy as np
import xgboost as xgb
import lightgbm as lgb


import matplotlib
import matplotlib.pyplot as plt


In [2]:

#удаляем лишние параметры и те, в которых нет данных
def change_frame(data):
    id = data[['source_id','original_ext_source_id', 'sdssdr9_oid', 'sdssdr9_oid.1','solution_id', 'designation']]
    #data1 = data.drop(columns = ['source_id', 'source_id.1', 'original_ext_source_id', 'sdssdr9_oid', 'sdssdr9_oid.1', 'solution_id', 'designation'])
    '''data1 = data.drop(columns = ['number_of_neighbours','number_of_mates','best_neighbour_multiplicity', 'astrometric_n_obs_al',
                                        'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al','astrometric_gof_al',
                                        'astrometric_chi2_al','astrometric_excess_noise','astrometric_excess_noise_sig',
                                        'astrometric_params_solved','astrometric_primary_flag','astrometric_weight_al',
                                        'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error','mean_varpi_factor_al',
                                        'astrometric_matched_observations','visibility_periods_used','astrometric_sigma5d_max',
                                        'frame_rotator_object_type','matched_observations','duplicated_source','phot_g_n_obs',
                                        'phot_bp_n_obs' ,'phot_rp_n_obs','radial_velocity','radial_velocity_error','rv_nb_transits',
                                        'rv_template_teff','rv_template_logg', 'rv_template_fe_h','phot_variable_flag','l','b','ecl_lon','ecl_lat','priam_flags',
                                        'teff_val','teff_percentile_lower','teff_percentile_upper','a_g_val','a_g_percentile_lower',
                                        'a_g_percentile_upper','e_bp_min_rp_val','e_bp_min_rp_percentile_lower','e_bp_min_rp_percentile_upper',
                                        'flame_flags','radius_val','radius_percentile_lower','radius_percentile_upper','lum_val','lum_percentile_lower',
                                        'lum_percentile_upper','datalink_url','epoch_photometry_url','obj_id','thing_id','clean_flag'])
    '''
    data1 = data[['source_id', 'source_id.1', 'original_ext_source_id', 'sdssdr9_oid', 'sdssdr9_oid.1','ra','dec','ra.1','dec.1','phot_g_mean_flux', 'phot_bp_mean_flux', 'phot_rp_mean_flux', 'u_mag', 'g_mag', 'z_mag', 'r_mag', 'i_mag']]
    data1 = data1.drop(columns = data1.columns[data1.isna().all()].tolist())
    data1 = data1.dropna()
    print(len(data1))
    return id, data1

In [ ]:
#графики

def barplot(x_data, y_data, error_data, x_label="", y_label="", title="", cnt_l = 40, cnt_r = 15):
    fig, ax = plt.subplots(figsize = (cnt_l, cnt_r))
    # Draw bars, position them in the center of the tick mark on the x-axis
    ax.bar(x_data, y_data, color = '#539caf', align = 'center', width = 0.9)
    # Draw error bars to show standard deviation, set ls to 'none'
    # to remove line between points
    ax.errorbar(x_data, y_data, color = '#297083', ls = 'none', lw = 2, capthick = 2)
    ax.set_ylabel(y_label)
    ax.set_xlabel(x_label)
    ax.set_title(title)

    
    plt.show()

In [ ]:
#расчет евклидового расстояния
def euclid_distance(data, x = '', y ='.1'):
    data['eucl'] = numpy.sqrt((data['ra' + x] - data['ra' + y])**2 + (data['dec' + x] - data['dec' + y])**2) 
    return data

Загрузка и предобработка данных

In [ ]:
# Пример загрузки данных
#without_match_x.csv - файлы пар разных объектов (несопоставимые)

no_data = pd.DataFrame()
for i in ['5','6']: 
    no_match_data = pd.read_csv("without_match_sdss_" + i + ".csv")
    no_data = pd.concat([no_data, no_match_data], axis = 0)
no_data = no_data.drop_duplicates()
no_match_id, no_data = change_frame(no_data)
no_data = no_data.drop_duplicates()

In [ ]:
#аналогично подгружаем сопоставимые пары
match_data = pd.DataFrame()
for i in range(1, 16): #max - 16
    data = pd.read_csv("with_" + str(i) + ".csv")
    match_data = pd.concat([match_data, data], axis = 0).drop_duplicates()

match_id, match_data_t = change_frame(match_data_t)

In [ ]:
#будущий у
match_data['matched'] = 1
no_data['matched'] = 0

In [ ]:
#создаем общий набор данных
df = pd.concat([match_data, no_data], axis = 0)
#отбираем нужные признаки
df = df[['ra','dec','ra.1','dec.1','phot_g_mean_flux', 'phot_bp_mean_flux', 'phot_rp_mean_flux', 'u_mag', 'g_mag', 'z_mag', 'r_mag', 'i_mag', 'matched']]

Создаем обучающий и тестовый наборы

In [ ]:
df = df.dropna()
y = df['matched']
df = df.drop(columns= ['matched'])
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.8, random_state=80, shuffle = True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.8, random_state=80, shuffle = True)


Работа методов машинного обучения

In [ ]:
dt = DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
res = dt.predict(X_val)

print('accuracy_score: ', accuracy_score(y_val, res))
print('precision_score: ', precision_score(y_val, res))
print('recall_score: ', recall_score(y_val, res))
print('f1_score: ', f1_score(y_val, res))
print()

print(dt.n_features_)
weight_f = dt.feature_importances_
print(weight_f)

col_lab = []
for i in X_train.columns:
    col_lab.append(i)

barplot(col_lab, weight_f, '', '', '', 'Вес признаков DT 1', 20, 30)

In [ ]:
res = dt.predict(X_test)

print('accuracy_score: ', accuracy_score(y_test, res))
print('precision_score: ', precision_score(y_test, res))
print('recall_score: ', recall_score(y_test, res))
print('f1_score: ', f1_score(y_test, res))
print()

In [ ]:
svc = SVC(verbose = True, random_state = 24)
svc = svc.fit(X_train, y_train)
res = svc.predict(X_val)

print('accuracy_score: ', accuracy_score(y_val, res))
print('precision_score: ', precision_score(y_val, res))
print('recall_score: ', recall_score(y_val, res))
print('f1_score: ', f1_score(y_val, res))
print()

res = svc.predict(X_test)
print('accuracy_score: ', accuracy_score(y_test, res))
print('precision_score: ', precision_score(y_test, res))
print('recall_score: ', recall_score(y_test, res))
print('f1_score: ', f1_score(y_test, res))
print()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3, algorithm='auto', leaf_size=30)
knn = knn.fit(X_train, y_train)
res = knn.predict(X_val)

print('accuracy_score: ', accuracy_score(y_val, res))
print('precision_score: ', precision_score(y_val, res))
print('recall_score: ', recall_score(y_val, res))
print('f1_score: ', f1_score(y_val, res))
print()

res = knn.predict(X_test)

print('accuracy_score: ', accuracy_score(y_test, res))
print('precision_score: ', precision_score(y_test, res))
print('recall_score: ', recall_score(y_test, res))
print('f1_score: ', f1_score(y_test, res))
print()

In [ ]:
model = xgb.XGBClassifier()
model = model.fit(X_train, y_train)
res_xgb = model.predict(X_val)
res = res_xgb

print('accuracy_score: ', accuracy_score(y_val, res))
print('precision_score: ', precision_score(y_val, res))
print('recall_score: ', recall_score(y_val, res))
print('f1_score: ', f1_score(y_val, res))
print()

res_xgb = model.predict(X_test)
res = res_xgb
print('accuracy_score: ', accuracy_score(y_test, res))
print('precision_score: ', precision_score(y_test, res))
print('recall_score: ', recall_score(y_test, res))
print('f1_score: ', f1_score(y_test, res))
print()

In [ ]:
m = len(X_train.columns)
p = len(y_train.drop_duplicates())
print(m)
print(p)
mlp = MLPClassifier(hidden_layer_sizes=((m+p)*100,), max_iter=200, verbose = True)
mlp.fit(X_train, y_train)
res = mlp.predict(X_val)

print('accuracy_score: ', accuracy_score(y_val, res))
print('precision_score: ', precision_score(y_val, res))
print('recall_score: ', recall_score(y_val, res))
print('f1_score: ', f1_score(y_val, res))
print()

In [ ]:
x, xe, y1, ye = train_test_split(X_train, y_train, test_size=0.35, random_state=22, )
model1 = lgb.LGBMClassifier(n_estimators=100, silent=True,  num_leaves = 25, learning_rate = 0.03, random_state = 27)
model1 = model1.fit(x, y1,   eval_set=[(xe, ye)], early_stopping_rounds=100)
res_lgb = model1.predict(X_val)
res = res_lgb

print('accuracy_score: ', accuracy_score(y_val, res))
print('precision_score: ', precision_score(y_val, res))
print('recall_score: ', recall_score(y_val, res))
print('f1_score: ', f1_score(y_val, res))
print()

In [ ]:
dd = euclid_distance(v1)
dd['eucl']

#разбиваем данные по заданному порогу
dd['match'] =  [1 if x < 0.005 else 0 for x in dd['eucl']] #0.00001 - не все правильные отметил 0.01 - почти все

In [ ]:
print('accuracy_score: ', accuracy_score(dd['matched'], dd['match']))
print('precision_score: ', precision_score(dd['matched'], dd['match']))
print('recall_score: ', recall_score(dd['matched'], dd['match']))
print('f1_score: ', f1_score(dd['matched'], dd['match']))
print()

Так как данные sdss оказались очень хорошими, то более детальное формирование выборок и проведения дополнительных экспериментов в данном файле нет
